# Goal: Scrape the JIRA API to obtain the changelog history for an issue

In [2]:
from jira import JIRA

In [3]:
# Connect to JIRA
jira = JIRA('https://jira.hulu.com',basic_auth=('**username**', '**password**'))

/Users/david.wang/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/jira/client.py:282: UserWarning: You are running an outdated version of JIRA Python 1.0.3. Current version is 1.0.7. Do not file any bugs against older versions.
  __version__, released_version))


## Import ticket keys via CSV (originally downloaded from JIRA, itself)

In [7]:
import csv

ticket_list = []

with open("/Volumes/home/w/wang.david/ticket_data_081116.csv", "rb") as csvfile:
    reader = csv.reader(csvfile, delimiter = '\t')
    for i in enumerate(reader):
        ticket_list.append(i[1][0])
print ticket_list[0:50]

['RR-7097', 'RR-7096', 'RR-7095', 'RR-7094', 'RR-7093', 'RR-7092', 'RR-7091', 'RR-7090', 'RR-7089', 'RR-7088', 'RR-7087', 'RR-7086', 'RR-7085', 'RR-7084', 'RR-7083', 'RR-7082', 'RR-7081', 'RR-7080', 'RR-7079', 'RR-7078', 'RR-7077', 'RR-7076', 'RR-7075', 'RR-7074', 'RR-7073', 'RR-7072', 'RR-7070', 'RR-7069', 'RR-7068', 'RR-7067', 'RR-7066', 'RR-7065', 'RR-7064', 'RR-7063', 'RR-7062', 'RR-7061', 'RR-7060', 'RR-7059', 'RR-7058', 'RR-7057', 'RR-7056', 'RR-7055', 'RR-7054', 'RR-7052', 'RR-7050', 'RR-7049', 'RR-7048', 'RR-7046', 'RR-7045', 'RR-7044']


## Create initial data structures for Table 1 for Stage 1 and 1.5
### Fields: Day created, status from, status to, story points, epic, sprint , resource

In [8]:
# Table 1 for Stage 1 and 1.5
# Loop through tickets and store info into dictionary
ticket_dict = {}

for ticket in ticket_list:
    # open the changelog
    issue_changelog = jira.issue(ticket, expand='changelog')
    issue = jira.issue(ticket)

    # assign changelog variable
    changelog = issue_changelog.changelog

    # Iterate through the history of change log and grab dates of changes
    for history in changelog.histories:
        for item in history.items:
            if item.field == 'status':
                # By setting it to =, it makes sure to always assign the last update
                ticket_dict[ticket] = ([history.created, item.fromString, item.toString])
                
    if ticket in ticket_dict:
        # Append Story Points
        if issue.fields.customfield_10262 is None:
            ticket_dict[ticket].append(0)
        else:
            ticket_dict[ticket].append(issue.fields.customfield_10262)
        # Append Epic
        if issue.fields.customfield_12431 is None:
            ticket_dict[ticket].append('none')
        else:
            ticket_dict[ticket].append(jira.issue(issue.fields.customfield_12431.encode('ascii','ignore')).fields.customfield_12432.encode('ascii','ignore'))
        # Append Sprint
        if issue.fields.customfield_11230 is None:
            ticket_dict[ticket].append(0.0)
        else:
            try:
                ticket_dict[ticket].append(float(issue.fields.customfield_11230[0].split("startDate",1)[0][-4:-1]))
            except(ValueError):
                ticket_dict[ticket].append(0.0)
        # Append Resource
        if issue.fields.assignee is None:
            ticket_dict[ticket].append('none')
        else:
            ticket_dict[ticket].append(issue.fields.assignee)

print 'done'

done


In [9]:
print jira.issue(jira.issue('RR-6853').fields.customfield_12431.encode('ascii','ignore')).fields.customfield_12432.encode('ascii','ignore')
        

pi6_mre_testing_overall (Curiosity/PPP)


In [10]:
print ticket_dict['RR-6835']

[u'2016-08-05T12:29:22.000-0700', u'Code Review', u'Test', 0, 'pi6_mre_report_development (Curiosity/PPP)', 5.5, <JIRA User: displayName=u'Daniel Baubeta', key=u'daniel.baubeta', name=u'daniel.baubeta'>]


## Create initial data structures Table 2 for Stage 2
### Fields: Day created, changelog history, status from, status to, epic, sprint , resource

In [11]:
# Table 2 for Stage 2
# Loop through tickets and store info into dictionary
ticket_dict_2 = {}

for ticket in ticket_list:
    # open the changelog
    issue_changelog = jira.issue(ticket, expand='changelog')
    issue = jira.issue(ticket)

    # assign changelog variable
    changelog = issue_changelog.changelog

    # Iterate through the history of change log and grab dates of changes
    for history in changelog.histories:
        for item in history.items:
            if item.field == 'status':
                if ticket in ticket_dict_2:
                    ticket_dict_2[ticket].append([history.created, item.fromString, item.toString])
                else:
                    # Created == Backlog (i.e. when a ticket is created, it is auto put into backlog)
                    ticket_dict_2[ticket] = [[issue.fields.created, 'Created', 'Backlog']]
                    # Backlog to Open
                    ticket_dict_2[ticket].append([history.created, item.fromString, item.toString])

    if ticket in ticket_dict:
        # Append Epic
        if issue.fields.customfield_12431 is None:
            ticket_dict_2[ticket].append('none')
        else:
            ticket_dict_2[ticket].append(jira.issue(issue.fields.customfield_12431.encode('ascii','ignore')).fields.customfield_12432.encode('ascii','ignore'))
        # Append Sprint
        if issue.fields.customfield_11230 is None:
            ticket_dict_2[ticket].append(0.0)
        else:
            try:
                ticket_dict_2[ticket].append(float(issue.fields.customfield_11230[0].split("startDate",1)[0][-4:-1]))
            except:
                ticket_dict_2[ticket].append(0.0)
        # Append Resource
        if issue.fields.assignee is None:
            ticket_dict_2[ticket].append('none')
        else:
            ticket_dict_2[ticket].append(issue.fields.assignee)
        # Append Summary
        if issue.fields.summary is None:
            ticket_dict_2[ticket].append('none')
        else:
            ticket_dict_2[ticket].append(issue.fields.summary)
        
print 'done'

done


In [12]:
print issue.fields.summary

MoneyTree QuarterEnd 2015Q4


In [13]:
test = []
for ticket in ticket_list:
    issue_changelog = jira.issue(ticket, expand='changelog')
    issue = jira.issue(ticket)
    try:
        test.append(jira.issue(issue.fields.customfield_12431.encode('ascii','ignore')).fields.summary.encode('ascii','ignore'))
    except:
        pass
    
for i in test:
    print i
    

KeyboardInterrupt: 

In [195]:
# Test/cross-check raw JIRA issues for correct values in the above scripts for table 1 and 2

issue = jira.issue('RR-6011')
print issue.fields.customfield_11230[0], '\n'
                                     
issue = jira.issue('RR-6772')
print issue.fields.customfield_11230[0], '\n'

line = float(issue.fields.customfield_11230[0].split("Reporting ",1)[1][0:3])
print type(line), '\n'

# Test script 1

# Test Table 1
print ticket_dict["RR-6800"], '\n'

# Test Table 2
print ticket_dict_2["RR-5831"], '\n'
print ticket_dict_2["RR-7024"]

com.atlassian.greenhopper.service.sprint.Sprint@3580d653[id=2399,rapidViewId=350,state=CLOSED,name=Metrics Reporting 5.1,startDate=2016-04-11T14:03:09.145-07:00,endDate=2016-04-24T14:03:00.000-07:00,completeDate=2016-04-25T13:32:18.091-07:00,sequence=2399] 

com.atlassian.greenhopper.service.sprint.Sprint@125c199c[id=2403,rapidViewId=350,state=CLOSED,name=Metrics Reporting 5.5,startDate=2016-06-06T13:41:35.898-07:00,endDate=2016-06-20T13:41:00.000-07:00,completeDate=2016-06-26T16:54:16.877-07:00,sequence=2402] 

<type 'float'> 

[u'2016-06-29T16:10:08.000-0700', u'Open', u'Closed', 3.0, 'PI5_MRE_Pipeline_ReArch (Curiosity/PPP)', 5.6, <JIRA User: displayName=u'Jenny Mehandru', key=u'jenny.mehandru', name=u'jenny.mehandru'>] 

[[u'2016-01-11T12:22:24.000-0800', 'Created', 'Backlog'], [u'2016-01-13T12:06:43.000-0800', u'Backlog', u'Open'], [u'2016-02-08T11:38:55.000-0800', u'Open', u'In Progress'], [u'2016-02-08T11:41:35.000-0800', u'In Progress', u'Code Review'], [u'2016-02-16T13:31:48.0

In [ ]:
# Test #2: output each individual item in each individual ticket for eyeball check

# Table 1
for ticket in ticket_dict:
    for item in ticket_dict[ticket]:
        print ticket, ": ", item
        

In [30]:
# Table 2
for ticket in ticket_dict_2:
    for item in ticket_dict_2[ticket]:
        print ticket, ": ", item

RR-5831 :  [u'2016-01-11T12:22:24.000-0800', 'Created', 'Backlog']
RR-5831 :  [u'2016-01-13T12:06:43.000-0800', u'Backlog', u'Open']
RR-5831 :  [u'2016-02-08T11:38:55.000-0800', u'Open', u'In Progress']
RR-5831 :  [u'2016-02-08T11:41:35.000-0800', u'In Progress', u'Code Review']
RR-5831 :  [u'2016-02-16T13:31:48.000-0800', u'Code Review', u'Test']
RR-5831 :  [u'2016-02-16T13:31:59.000-0800', u'Test', u'Resolved']
RR-5831 :  PI5_MRE_Workday_MoneyTreeChanges
RR-5831 :  4.2
RR-5831 :  Sakshi Mittal
RR-5831 :  Integration testing Workday
RR-5920 :  [u'2016-01-13T12:17:28.000-0800', 'Created', 'Backlog']
RR-5920 :  [u'2016-02-29T13:22:12.000-0800', u'Backlog', u'Closed']
RR-5920 :  none
RR-5920 :  4.3
RR-5920 :  none
RR-5920 :  Identify Sync Process
RR-6011 :  [u'2016-01-13T13:44:22.000-0800', 'Created', 'Backlog']
RR-6011 :  [u'2016-01-13T16:01:26.000-0800', u'Backlog', u'Open']
RR-6011 :  [u'2016-04-19T11:49:40.000-0700', u'Open', u'In Progress']
RR-6011 :  [u'2016-04-22T11:52:59.000-0700

## Define functions to convert unicode datetime and to calculate duration
### duration = time (measured in days) it took to complete a certain status ("In Progress" for Ticket RR-5911 took 5 days")

In [14]:
# Test #3 - time fields
issue = jira.issue('RR-7007')
print issue.fields.created

2016-07-05T17:11:17.000-0700


In [15]:
# date-time parser and converter
## convert unicode datetime to python readable datetime

from datetime import datetime

def convert_datetime(u_time):
    date_posted = str(u_time)[:-5]+'Z'
    return datetime.strptime(date_posted, '%Y-%m-%dT%H:%M:%S.%fZ') 

def duration(d1, d2):
    durat = float((d2 - d1).days)
    # if the duration = 0, set it to 0.5 since a 0 meaning would be incorrect on Tableau
    # this is the hacky way to do it until we find out how to calculate half days or minutes
    if durat == 0.0:
        durat = 0.5
    return durat

# Unsure duration can support minutes or seconds...

In [16]:
# test convert_datetime function

time_test_1 = convert_datetime(ticket_dict_2["RR-7007"][0][0])
print time_test_1

# test duration function
time_test_2 = convert_datetime(ticket_dict_2["RR-7007"][1][0])
print time_test_2

print duration(time_test_1, time_test_2)

# Tests work!

2016-07-05 17:11:17
2016-07-05 17:11:29
0.5


## Getter functions to get the epic and sprint for a ticket

In [17]:
# function to get the sprint and epic for table 2 rows

def get_epic (ticket):
    list_object = ticket_dict_2[ticket]
    for item in list_object:
        # return the first item that is not a list to get the epic
        if type(item) != list:
            return item;
    
def get_sprint (ticket):
    list_object = ticket_dict_2[ticket]
    item_found = False
    for item in list_object:
        if type(item) == float:
            a = item
            item_found = True
            break
        else:
            b = 0.0
            item_found = False
    if item_found:
        return a
    else:
        return b
        
    

In [18]:
# Test getter functions above

print get_epic("RR-6159")
print get_sprint("RR-7007")
print type(str(ticket_dict_2["RR-7007"][-2]))



PI4_MRE_Pagerduty, alerting, and monitoring_Reporting_Work 
6.1
<type 'str'>


In [19]:
print ['https://jira.hulu.com/browse/' + ticket]

['https://jira.hulu.com/browse/RR-6811']


## Finish creating Table 1 for Stage 1 and 1.5

In [20]:
import pandas as pd
import re

In [21]:
table_1 = []

for ticket in ticket_dict:
    table_1.append([ticket] + [convert_datetime(ticket_dict[ticket][0])] + ticket_dict[ticket][1:7] + 
                  ['https://jira.hulu.com/browse/' + ticket])

print len(table_1[0])
    
# EB Test for table_1
df_2 = pd.DataFrame(table_1, columns = ['Ticket',
                                        'Date Last Changed',
                                        'Status From',
                                        'Status To',
                                        'Story Points',
                                        'Epic',
                                        'Sprint',
                                        'Resource',
                                        'URL'])
df_2

9


,Ticket,Date Last Changed,Status From,Status To,Story Points,Epic,Sprint,Resource,URL
0,RR-5920,2016-02-29 13:22:12,Backlog,Closed,0.0,none,4.3,none,https://jira.hulu.com/browse/RR-5920
1,RR-6011,2016-04-25 13:14:14,Code Review,Closed,2.0,PI5_MRE_Pipeline_ReArch (Curiosity/PPP),5.1,Peter Hsin,https://jira.hulu.com/browse/RR-6011
2,RR-6499,2016-07-05 10:12:54,In Progress,Closed,3.0,PI5_MRE_Pipeline_ReArch (Curiosity/PPP),5.5,Jenny Mehandru,https://jira.hulu.com/browse/RR-6499
3,RR-6498,2016-06-06 13:35:47,Code Review,Resolved,3.0,PI5_MRE_Pipeline_ReArch (Curiosity/PPP),5.4,Jenny Mehandru,https://jira.hulu.com/browse/RR-6498
4,RR-6497,2016-05-09 14:35:15,Code Review,Closed,3.0,PI5_MRE_Pipeline_ReArch (Curiosity/PPP),5.2,Jenny Mehandru,https://jira.hulu.com/browse/RR-6497
5,RR-6496,2016-04-06 18:40:24,Backlog,Open,5.0,PI5_MRE_Pipeline_ReArch (Curiosity/PPP),6.1,none,https://jira.hulu.com/browse/RR-6496
6,RR-6495,2016-05-03 11:24:49,Code Review,Closed,8.0,PI5_MRE_Pipeline_ReArch (Curiosity/PPP),5.1,Jenny Mehandru,https://jira.hulu.com/browse/RR-6495
7,RR-5925,2016-01-13 16:18:24,Backlog,Open,3.0,pi6_mre_remaining_dev_work (Curiosity/PPP),5.2,Ankush Sharma,https://jira.hulu.com/browse/RR-5925
8,RR-5923,2016-02-11 15:49:12,Open,Closed,1.0,PI4_MRE_Pipeline_ReArch (Curiosity/PPP),4.3,none,https://jira.hulu.com/browse/RR-5923
9,RR-6491,2016-04-06 16:25:19,Backlog,Open,1.0,PI5_MRE_NBC_CLA_feeds,5.5,none,https://jira.hulu.com/browse/RR-6491


In [22]:
print table_1[0]


['RR-5920', datetime.datetime(2016, 2, 29, 13, 22, 12), u'Backlog', u'Closed', 0, 'none', 4.3, 'none', 'https://jira.hulu.com/browse/RR-5920']


In [25]:
print ticket_dict_2['RR-6499']

[[u'2016-04-06T16:56:37.000-0700', 'Created', 'Backlog'], [u'2016-04-06T17:26:12.000-0700', u'Backlog', u'Open'], [u'2016-06-03T14:53:16.000-0700', u'Open', u'Code Review'], [u'2016-06-29T16:10:40.000-0700', u'Code Review', u'In Progress'], [u'2016-07-05T10:12:54.000-0700', u'In Progress', u'Closed'], 'PI5_MRE_Pipeline_ReArch (Curiosity/PPP)', 5.5, <JIRA User: displayName=u'Jenny Mehandru', key=u'jenny.mehandru', name=u'jenny.mehandru'>, u'PPP  - Resource Pool Training -Training Support Testing']


## Finish creating Table 2 for Stage 2
### Using data structures for Table 2, getter functions, date functions

In [26]:
duration_list = []
j = 1

for ticket in  ticket_dict_2:
    done = 0
    # get epic and sprint for the ticket and store into variables
    epic = get_epic(ticket)
    sprint = get_sprint(ticket)
    # for each item in the issue (of ticket dict 2)...
    for i in range (len(ticket_dict_2[ticket])-1, -1, -1):
        if type(ticket_dict_2[ticket][i]) == list and type(ticket_dict_2[ticket][i-1]) == list:
            next_date = convert_datetime(ticket_dict_2[ticket][i][0])
            current_date = convert_datetime(ticket_dict_2[ticket][i-1][0])
            durat = duration(current_date, next_date)
            summary = str(ticket_dict_2[ticket][-1].encode('utf8'))[:50]
            assignee = str(ticket_dict_2[ticket][-2])
            summ_regexed = re.sub(r'[^\w]', ' ', summary)
            url = 'https://jira.hulu.com/browse/' + ticket
            # categorial variable "done" to indicate if the ticket is finished (closed/resolved)
            if str(ticket_dict_2[ticket][i-1][2]) == 'Resolved' or str(ticket_dict_2[ticket][i-1][2]) == 'Closed':
                done = 1
                
            duration_list.append([j, ticket, ticket_dict_2[ticket][i-1][2], done, durat, current_date, next_date, 
                                  sprint, epic, assignee, summ_regexed, url])
            
        elif type(ticket_dict_2[ticket][i]) != list and type(ticket_dict_2[ticket][i-1]) == list:
            current_date = convert_datetime(ticket_dict_2[ticket][i-1][0])
            summary = str(ticket_dict_2[ticket][-1].encode('utf8'))[:50]
            assignee = str(ticket_dict_2[ticket][-2])
            summ_regexed = re.sub(r'[^\w]', ' ', summary)
            url = 'https://jira.hulu.com/browse/' + ticket
            if str(ticket_dict_2[ticket][i-1][2]) == 'Resolved' or str(ticket_dict_2[ticket][i-1][2]) == 'Closed':
                done = 1
            # append the last status and store duration to 1
            duration_list.append([j, ticket, ticket_dict_2[ticket][i-1][2], done, 1, current_date, current_date,
                                  get_sprint(ticket), get_epic(ticket), assignee, summ_regexed, url]) 
        else:
            continue
        j+=1

# EB test of data
df = pd.DataFrame(duration_list, columns=['id', 'Ticket', 'Status', 'Done', 'Duration', 'Start date', 'End date', 
                                          'Sprint', 'Epic', 'Resource', 'Summary', 'URL'])
df

,id,Ticket,Status,Done,Duration,Start date,End date,Sprint,Epic,Resource,Summary,URL
0,1,RR-5920,Closed,1,1.0,2016-02-29 13:22:12,2016-02-29 13:22:12,4.3,none,none,Identify Sync Process,https://jira.hulu.com/browse/RR-5920
1,2,RR-5920,Backlog,1,47.0,2016-01-13 12:17:28,2016-02-29 13:22:12,4.3,none,none,Identify Sync Process,https://jira.hulu.com/browse/RR-5920
2,3,RR-6011,Closed,1,1.0,2016-04-25 13:14:14,2016-04-25 13:14:14,5.1,PI5_MRE_Pipeline_ReArch (Curiosity/PPP),Peter Hsin,AccountEntry Design Aggregations,https://jira.hulu.com/browse/RR-6011
3,4,RR-6011,Code Review,1,3.0,2016-04-22 11:52:59,2016-04-25 13:14:14,5.1,PI5_MRE_Pipeline_ReArch (Curiosity/PPP),Peter Hsin,AccountEntry Design Aggregations,https://jira.hulu.com/browse/RR-6011
4,5,RR-6011,In Progress,1,3.0,2016-04-19 11:49:40,2016-04-22 11:52:59,5.1,PI5_MRE_Pipeline_ReArch (Curiosity/PPP),Peter Hsin,AccountEntry Design Aggregations,https://jira.hulu.com/browse/RR-6011
5,6,RR-6011,Open,1,96.0,2016-01-13 16:01:26,2016-04-19 11:49:40,5.1,PI5_MRE_Pipeline_ReArch (Curiosity/PPP),Peter Hsin,AccountEntry Design Aggregations,https://jira.hulu.com/browse/RR-6011
6,7,RR-6011,Backlog,1,0.5,2016-01-13 13:44:22,2016-01-13 16:01:26,5.1,PI5_MRE_Pipeline_ReArch (Curiosity/PPP),Peter Hsin,AccountEntry Design Aggregations,https://jira.hulu.com/browse/RR-6011
7,8,RR-6499,Closed,1,1.0,2016-07-05 10:12:54,2016-07-05 10:12:54,5.5,PI5_MRE_Pipeline_ReArch (Curiosity/PPP),Jenny Mehandru,PPP Resource Pool Training Training Suppor...,https://jira.hulu.com/browse/RR-6499
8,9,RR-6499,In Progress,1,5.0,2016-06-29 16:10:40,2016-07-05 10:12:54,5.5,PI5_MRE_Pipeline_ReArch (Curiosity/PPP),Jenny Mehandru,PPP Resource Pool Training Training Suppor...,https://jira.hulu.com/browse/RR-6499
9,10,RR-6499,Code Review,1,26.0,2016-06-03 14:53:16,2016-06-29 16:10:40,5.5,PI5_MRE_Pipeline_ReArch (Curiosity/PPP),Jenny Mehandru,PPP Resource Pool Training Training Suppor...,https://jira.hulu.com/browse/RR-6499


In [27]:
for index, row in df.iterrows():
    if row['Done'] == 0:
        print row['Ticket']



RR-6496
RR-6496
RR-5925
RR-5925
RR-6491
RR-6491
RR-6252
RR-6252
RR-6252
RR-6311
RR-6311
RR-6311
RR-6312
RR-6312
RR-6312
RR-6315
RR-6315
RR-6844
RR-6844
RR-6844
RR-6844
RR-6845
RR-6845
RR-6845
RR-6845
RR-6969
RR-6969
RR-6968
RR-6968
RR-6840
RR-6840
RR-6840
RR-6840
RR-6840
RR-6841
RR-6841
RR-6841
RR-6841
RR-6841
RR-6842
RR-6842
RR-6842
RR-6842
RR-6843
RR-6843
RR-6843
RR-6843
RR-6843
RR-6963
RR-6963
RR-6962
RR-6962
RR-6961
RR-6961
RR-6960
RR-6960
RR-6967
RR-6967
RR-6966
RR-6966
RR-6965
RR-6965
RR-6964
RR-6964
RR-7032
RR-7032
RR-7033
RR-7033
RR-7030
RR-7030
RR-7031
RR-7031
RR-7035
RR-7035
RR-6404
RR-6404
RR-6406
RR-6406
RR-6406
RR-6406
RR-6406
RR-6406
RR-6406
RR-6406
RR-6406
RR-6407
RR-6407
RR-6407
RR-6407
RR-6400
RR-6400
RR-6408
RR-6408
RR-6408
RR-6408
RR-6408
RR-6661
RR-6661
RR-6662
RR-6662
RR-6662
RR-6662
RR-6662
RR-6662
RR-6663
RR-6663
RR-6663
RR-6663
RR-6663
RR-6663
RR-6665
RR-6665
RR-6665
RR-6665
RR-6665
RR-6666
RR-6666
RR-6666
RR-6666
RR-6666
RR-7048
RR-7048
RR-7048
RR-7048
RR-6608


In [28]:
s = "Give Daphne & Alison 'edit ability for reporting "
re.sub(r'[^\w]', ' ', s)

print str(ticket_dict_2['RR-5831'][5][2])

KeyError: 'RR-5831'

In [29]:
# Tests for above scripts

print ticket_dict["RR-6011"], '\n'
print ticket_dict_2["RR-6011"], '\n'
print ticket
print next_date
print len(ticket_dict_2[ticket]) 
print type(ticket_dict_2[ticket][i][2])

[u'2016-04-25T13:14:14.000-0700', u'Code Review', u'Closed', 2.0, 'PI5_MRE_Pipeline_ReArch (Curiosity/PPP)', 5.1, <JIRA User: displayName=u'Peter Hsin', key=u'pete.hsin', name=u'pete.hsin'>] 

[[u'2016-01-13T13:44:22.000-0800', 'Created', 'Backlog'], [u'2016-01-13T16:01:26.000-0800', u'Backlog', u'Open'], [u'2016-04-19T11:49:40.000-0700', u'Open', u'In Progress'], [u'2016-04-22T11:52:59.000-0700', u'In Progress', u'Code Review'], [u'2016-04-25T13:14:14.000-0700', u'Code Review', u'Closed'], 'PI5_MRE_Pipeline_ReArch (Curiosity/PPP)', 5.1, <JIRA User: displayName=u'Peter Hsin', key=u'pete.hsin', name=u'pete.hsin'>, u'AccountEntry - Design - Aggregations'] 

RR-6559
2016-04-26 17:32:49
8
<type 'str'>


# Highlighting problem tickets
## Goal: Create separate lists and just append them during csv creation

In [30]:
import datetime

# function to find the end date for a sprint
def find_end_date(sprint):    
    ## calculate the int difference between the two sprint cycles
    ## base case is sprint 6.1 with a start date of July 4th, 2016, and end date of July 15th, 2016
    
    ## get the base_day (set to the end date of 6.1) and adjuster (each sprint lasts 14 days)
    adjust = datetime.timedelta(days = 14)
    base_day = datetime.date(2016,7,15)
    
    ## Final time calculations
    if sprint < 6.1:
        raw_calc = 61 - (sprint * 10)
        sprint_ones = int(str(sprint)[0])
        diff_adjust = 4 * (6 - sprint_ones)
        final_diff = int(raw_calc - diff_adjust)
        end_date = base_day - adjust * final_diff
    elif sprint > 6.1:
        # swap equation spots in raw_calc and diff_adjust
        raw_calc = (sprint * 10) - 61
        sprint_ones = int(str(sprint)[0])
        diff_adjust = 4 * (sprint_ones - 6)
        final_diff = int(raw_calc - diff_adjust)
        end_date = base_day + adjust * final_diff
    elif sprint == 6.1:
        end_date = base_day
    else:
        # some distant time in the future...
        end_date = datetime.date(9999,12,31)
    
    return end_date

# test find_end_date function
print find_end_date(4.1)
print find_end_date(4.4)
print find_end_date(5.3)
print find_end_date(5.6)
print find_end_date(6.1)
print find_end_date(6.2)
print find_end_date(6.3)
print find_end_date(6.4)
print find_end_date(6.5)
print find_end_date(6.6)

2016-01-29
2016-03-11
2016-05-20
2016-07-01
2016-07-15
2016-07-29
2016-08-12
2016-08-26
2016-09-09
2016-09-23


## Current highlight cases as identified by the function below:
### 1) If the ticket is not done, and the ticket's story points is greater than the difference between today and the end date of the sprint --> highlight the ticket (aka put it into the problem tickets list)
### 2) ...more to come

In [31]:
# Main script
## to be appended to table 2/3

raw_problem_tickets = []

for ticket_row in duration_list:
    # if the ticket is not done, and the ticket's story points > difference between today and end of sprint...
    if ticket_row[3] != 1 and ticket_dict[ticket_row[1]][3] > duration(datetime.date.today(), find_end_date(ticket_row[7])):
        raw_problem_tickets.append(ticket_row[1])
    else:
        continue
        
unique_problem_tickets = list(set(raw_problem_tickets))

In [32]:
print unique_problem_tickets

['RR-6302', 'RR-6394', 'RR-6395', 'RR-6152', 'RR-6153', 'RR-6674', 'RR-6154', 'RR-7025', 'RR-7027', 'RR-7026', 'RR-7023', 'RR-7022', 'RR-6496', 'RR-5925', 'RR-7029', 'RR-5951', 'RR-6491', 'RR-6609', 'RR-6301', 'RR-6252', 'RR-6410', 'RR-6311', 'RR-6312', 'RR-6315', 'RR-6844', 'RR-6845', 'RR-6846', 'RR-6840', 'RR-6841', 'RR-6842', 'RR-6843', 'RR-6962', 'RR-6961', 'RR-6960', 'RR-6682', 'RR-6966', 'RR-6680', 'RR-6681', 'RR-6671', 'RR-6325', 'RR-7033', 'RR-7030', 'RR-7031', 'RR-6778', 'RR-6676', 'RR-6675', 'RR-7035', 'RR-6127', 'RR-6831', 'RR-6679', 'RR-5950', 'RR-6776', 'RR-6777', 'RR-6404', 'RR-6406', 'RR-6407', 'RR-6400', 'RR-6577', 'RR-6408', 'RR-6572', 'RR-6978', 'RR-6870', 'RR-6873', 'RR-6872', 'RR-6875', 'RR-6874', 'RR-6876', 'RR-6970', 'RR-6976', 'RR-6977', 'RR-6332', 'RR-6661', 'RR-6662', 'RR-6663', 'RR-6665', 'RR-6666', 'RR-6673', 'RR-7028', 'RR-6945', 'RR-6946', 'RR-6940', 'RR-6942', 'RR-6230', 'RR-6949', 'RR-6948', 'RR-7048', 'RR-6608', 'RR-6868', 'RR-6869', 'RR-6866', 'RR-6867'

## Search through problem_tickets, and flag the problem tickets in table 1 and 3

In [33]:
# Table 1 identify problem tickets and append categorical 1

for ticket_list in table_1:
    if ticket_list[1] in unique_problem_tickets:
        ticket_list.append(1)
    else:
        ticket_list.append(0)

In [34]:
# Table 2/3 identify problem tickets and append categorical 1

for ticket_list in duration_list:
    if ticket_list[1] in unique_problem_tickets:
        ticket_list.append(1)
    else:
        ticket_list.append(0)

## Write it into a CSV file
### Destination path is server smb://share1/home/w/wang.david

In [35]:
# write results to CSV File

# Table_1

import csv

with open("/Volumes/home/w/wang.david/Table_1.csv", "wb") as csvfile:
    ticket_writer = csv.writer(csvfile)
    ticket_writer.writerow(['Ticket', 'Date Last Changed', 'Status From', 'Status To', 'Story Points', 
                            'Epic', 'Sprint', 'Resource', 'URL', 'Problem'])
    for row in table_1:
        ticket_writer.writerow(row)

In [36]:
# Table_2 AKA Table_3

with open("/Volumes/home/w/wang.david/Table_3.csv", "wb") as csvfile:
    ticket_writer = csv.writer(csvfile)
    ticket_writer.writerow(['id', 'Ticket', 'Status', 'Done', 'Duration', 'Start date', 
                            'End date', 'Sprint', 'Epic', 'Resource', 'Summary', 'URL', 'Problem'])
    for row in duration_list:
        ticket_writer.writerow(row)